In [ ]:
import math
import os

import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from modules.vit_model import vit_mt

modelid = 'condition'
gpu = 0
# create model
model = vit_mt()
torch.cuda.set_device(gpu)
model = model.cuda(gpu)
loc = 'cuda:{}'.format(gpu)
checkpoint = torch.load(f'data/{modelid}.tar', map_location=loc)
model.load_state_dict(checkpoint['state_dict'])
del checkpoint

In [ ]:
bs = 4

import json
import torchvision.transforms as transforms
import cv2


trans = transforms.Compose([            
    transforms.ToTensor(),
    transforms.Resize(384),
    transforms.CenterCrop(384),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])



In [ ]:
finds = []
start_dir = '../da_pairs'
listdir = os.listdir(start_dir)
list0 = []
for entry in listdir:
    name_folder = os.path.join(start_dir,entry)
    photo_dir = os.listdir( name_folder )
    list0.append({'name':entry, 'pnumbers':len(photo_dir),'photos':list(photo_dir)})


In [ ]:
handlers = []
saved_feature = []

def features_hook(module, input, output):
    x = torch.stack(tuple(output))
    saved_feature.append(output)

module = model.img_org.norm
handlers.append(module.register_forward_hook(features_hook))

In [ ]:
items = [os.path.join(start_dir,x['name'],y) for x in list0 for y in x['photos'] ]

print(len(items))

In [ ]:
total_batch = len(items)//bs + 1

model.eval()

if not os.path.exists(f'data/{modelid}'):
     os.mkdir(f'data/{modelid}')
            
for batch in range(total_batch):
    this_end = min((batch+1)* bs, len(items))
    
    paths = items[batch*bs:this_end]
    images = []
    for p in paths:
        p1 = cv2.imread(p)
        p1 = cv2.cvtColor(p1,cv2.COLOR_BGR2RGB)
        p1 = trans(p1)
        images.append(p1)
    images = torch.stack(images).to(loc)
    with torch.no_grad():
        fcs,pred1 = model(images)

    fcs = saved_feature[-1]
    fcs = fcs.detach().cpu().numpy()

    saved_feature = []

    for i in range(len(paths)):
        source = paths[i]
        fc = fcs[i,:]
        newfile = source.replace('.png','').replace('../da_pairs',f'data/{modelid}')
        if not os.path.exists(os.path.dirname(newfile)):
            os.mkdir(os.path.dirname(newfile))
        np.save(newfile,fc)
    print(batch,'of',total_batch)